In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [7]:
path = "../training/conf/network/conv_transformer.yaml"

In [35]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [36]:
cfg

{'_target_': 'text_recognizer.networks.ConvTransformer', 'input_dims': [1, 1, 576, 640], 'hidden_dim': 144, 'num_classes': 58, 'pad_index': 3, 'encoder': {'_target_': 'text_recognizer.networks.EfficientNet', 'arch': 'b0', 'stochastic_dropout_rate': 0.2, 'bn_momentum': 0.99, 'bn_eps': 0.001, 'depth': 5, 'out_channels': 144}, 'decoder': {'_target_': 'text_recognizer.networks.transformer.Decoder', 'depth': 6, 'block': {'_target_': 'text_recognizer.networks.transformer.DecoderBlock', 'self_attn': {'_target_': 'text_recognizer.networks.transformer.Attention', 'dim': 144, 'num_heads': 12, 'dim_head': 64, 'dropout_rate': 0.2, 'causal': True, 'rotary_embedding': {'_target_': 'text_recognizer.networks.transformer.RotaryEmbedding', 'dim': 64}}, 'cross_attn': {'_target_': 'text_recognizer.networks.transformer.Attention', 'dim': 144, 'num_heads': 8, 'dim_head': 64, 'dropout_rate': 0.2, 'causal': False}, 'norm': {'_target_': 'text_recognizer.networks.transformer.RMSNorm', 'dim': 144}, 'ff': {'_targ

In [37]:
net = instantiate(cfg)

In [38]:
from torchinfo import summary

In [39]:
summary(net, ((1, 1, 56, 1024), (1, 89)), device="cpu", depth=4)

torch.Size([1, 144, 3, 63])


Layer (type:depth-idx)                                       Output Shape              Param #
ConvTransformer                                              [1, 58, 89]               --
├─EfficientNet: 1-1                                          [1, 144, 3, 63]           850,880
│    └─Sequential: 2-1                                       [1, 32, 26, 510]          --
│    │    └─ZeroPad2d: 3-1                                   [1, 1, 57, 1025]          --
│    │    └─Conv2d: 3-2                                      [1, 32, 26, 510]          1,568
│    │    └─BatchNorm2d: 3-3                                 [1, 32, 26, 510]          64
│    │    └─Mish: 3-4                                        [1, 32, 26, 510]          --
│    └─ModuleList: 2                                         --                        --
│    │    └─MBConvBlock: 3-5                                 [1, 16, 26, 510]          --
│    │    │    └─Depthwise: 4-1                              [1, 32, 26, 510]          

In [22]:
summary(net, ((1, 1, 576, 640), (1, 682)), device="cpu", depth=4)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [EfficientNet: 1, Sequential: 2, ZeroPad2d: 3, Conv2d: 3, BatchNorm2d: 3, Mish: 3, MBConvBlock: 3, Depthwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, Mish: 6, SqueezeAndExcite: 4, Sequential: 5, Conv2d: 6, Mish: 6, Conv2d: 6, Pointwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, MBConvBlock: 3, InvertedBottleneck: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, Mish: 6, Depthwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, Mish: 6, SqueezeAndExcite: 4, Sequential: 5, Conv2d: 6, Mish: 6, Conv2d: 6, Pointwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, MBConvBlock: 3, InvertedBottleneck: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, Mish: 6, Depthwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, Mish: 6, SqueezeAndExcite: 4, Sequential: 5, Conv2d: 6, Mish: 6, Conv2d: 6, Pointwise: 4, Sequential: 5, Conv2d: 6, BatchNorm2d: 6, MBConvBlock: 3, InvertedBottleneck: 4, Sequential: 5, 